<a href="https://colab.research.google.com/github/Jocelyn-git/Python_ML/blob/main/kaggle_Spaceship_Titanic_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Random Forest 
#Observations

正相關性係數可能欄位：“Destination”, "Age", "VIP", "RoomService"  
 負相關性係數可能欄位：“PassengerId”, "Name", "CryoSleep", "Cabin", ”HomePlanet“ 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Loading data
Path_train = "./drive/MyDrive/Colab-Notebooks/spaceship-titanic/train.csv"
Path_test = "./drive/MyDrive/Colab-Notebooks/spaceship-titanic/test.csv"
train_data = pd.read_csv(Path_train)
test_data = pd.read_csv(Path_test)
test_ids = test_data["PassengerId"]

In [4]:
#Check data
train_data.info()
train_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
#Data Clean
#清除較不相關且非數值的自變數(X)

def clean(data):
    data = data.drop(["PassengerId", "Name", "CryoSleep", "Cabin", "HomePlanet"], axis=1)

    cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall","Spa","VRDeck"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    #data.Embarked.fillna("U", inplace=True)

    return data

In [6]:
train_data = clean(train_data)
test_data = clean(test_data)

train_data.head(5)

,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [7]:
#Logistic Regression
#觀察相關性(Y＝ Transported)
dummy_x_logi = pd.get_dummies(train_data.iloc[:,:-1])
train_y_logi = (train_data["Transported"]== True).astype(int)
#train_y_logi = train_y_logi.iloc[:]
x_test_logi  = pd.get_dummies(test_data.iloc[:,:])
#dummy_x_logi.head(5)
#x_test_logi.head(5)
#train_y_logi.head(5)

dummy_x_logi.shape, train_y_logi.shape, x_test_logi.shape

((8693, 11), (8693,), (4277, 11))

In [8]:
#評估模型並檢視信賴分數
logreg = LogisticRegression()
logreg.fit(dummy_x_logi,train_y_logi)
Y_pred = logreg.predict(x_test_logi)

acc_log = round(logreg.score(dummy_x_logi, train_y_logi) * 100, 2)
acc_log

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


76.96

In [9]:
#顯示相關性
coeff_df = pd.DataFrame(dummy_x_logi.columns)
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
6,Destination_55 Cancri e,0.676345
9,VIP_False,0.237493
10,VIP_True,0.064928
0,Age,0.004589
2,FoodCourt,0.000706
3,ShoppingMall,0.000244
1,RoomService,-0.002097
5,VRDeck,-0.002148
4,Spa,-0.002358
8,Destination_TRAPPIST-1e,-0.117361


In [10]:
dummy_x_logi.head(5)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,0,0,1,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,0,1,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,1,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,0,0,1,1,0


In [11]:
#Random Forest
#數據標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x_logi)

In [12]:
#建模並評估
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, train_y_logi, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [02:17<00:00,  2.86s/it]


In [13]:
depth = []
for i in range(2,100,1):
  tree = DecisionTreeClassifier(criterion="entropy", max_depth = i, random_state=10)
  scores = cross_val_score(tree, sc_x, train_y_logi, cv=3)
  depth.append(scores.mean())

In [14]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

7


In [15]:

estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=final_depth)
  scores = cross_val_score(rf, sc_x, train_y_logi, cv=3)
  estimators.append(scores.mean())

100%|██████████| 150/150 [03:22<00:00,  1.35s/it]


In [16]:
final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

112


In [17]:
final_rf = RandomForestClassifier(n_estimators=final_estimators, max_depth=final_depth)
final_rf.fit(sc_x, train_y_logi)

RandomForestClassifier(max_depth=7, n_estimators=112)

In [18]:
sc_test_data_x = scaler.transform(x_test_logi)

In [19]:
# Convert dummy value to True/False
prediction = final_rf.predict(sc_test_data_x)
prediction = prediction.astype(str)
prediction = np.char.replace(prediction, '0', "False")
prediction = np.char.replace(prediction, '1', "True")

prediction

array(['True', 'False', 'True', ..., 'True', 'True', 'True'], dtype='<U5')

In [20]:
#Create Predit Data files
gender_submission = pd.DataFrame({"PassengerId": test_ids.values, "Transported": prediction })
gender_submission.to_csv("submission_rf.csv", index=False)